In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from math import nan
import dask

### Read in the coordinats from the SMYLE simulations to set up the reanalysis in the same way

In [11]:
smyle = xr.open_dataset("/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/outputzm/Uzm_BSMYLE-CW3E_day_init02.nc")
smyle = smyle.coords

In [12]:
ystart=smyle['init_year'].isel(init_year=0).values
yend=smyle['init_year'].isel(init_year=smyle['init_year'].size-1).values
nyears=yend-ystart+1
initmon=smyle['time'].isel(time=0).time.dt.month.values

### Set up the dask cluster

In [5]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '10GB',
    processes = 1,
    queue = 'casper',
    local_directory='$TMPDIR',
    resource_spec='select=1:ncpus=1:mem=10GB',
    project='P04010022',
    walltime='03:00:00',
    interface='ib0')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [6]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.60:33959,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
era5 = xr.open_mfdataset("/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/obs/ERA5/fluxes/*.nc")

In [8]:
era5 = era5.drop_vars(['S','col','row'])

### Accumulate in the form (init_year, day , level, lat)

In [14]:
dpm = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
smon = initmon
emon = (initmon - 1)
if (emon < 1):
    emon = 12    

sdatestr = str(smon).zfill(2)+'-01' ; edatestr = str(emon).zfill(2)+'-'+str(dpm[emon]).zfill(2)

era5out=[]

for iyear in np.arange(ystart,yend+1,1):
    print(iyear)
    eyear = iyear+1
    if (emon == 12):
        eyear = iyear

    era5use = era5.sel(time=slice(str(iyear)+'-'+sdatestr, str(eyear)+'-'+edatestr))
    
    # drop the 29th Feb on leap years
    era5use = era5use.sel(time=~((era5use.time.dt.month == 2) & (era5use.time.dt.day == 29)))
    era5use['time'] = smyle['time'].values
    era5out.append(era5use)
    
era5out = xr.concat(era5out, dim='init_year')
era5out = era5out.assign_coords(init_year = smyle['init_year'])

1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [15]:
era5out.to_netcdf("/glade/campaign/cgd/cas/islas/python/smyle_cw3e/DATA_SORT/outputzm/Uzm_ERA5_day_init"+str(initmon).zfill(2)+".nc")

In [16]:
cluster.close()